In [1]:
import csv
import re
import os
import pandas as pd
import numpy as np
import xlsxwriter
import shutil

from datetime import datetime, date

In [2]:
root_dir = '..'
file_root = f'{root_dir}/files'
church_export_root = f'{root_dir}/church'

pricing_breakdown = {
    'student': {
        '05-10-24': 200,
        '05-24-24': 230,
        '05-31-24': 275
    },
    'chaperone': {
        'High School Camp': 150,
        'Middle School Camp': 150,
        'Both Camps': 250,
    }
}

late_fee = 25

church_list = [
    "Ashland City Faith Community",
    "Bell Road",
    "Bent Knee Cowboy",
    "Body of Christ",
    "Bread of Life Community",
    "Bridge Point",
    "Brookhaven First",
    "Clarksdale",
    "Clarksville First Community of Hope",
    "Clarksville Grace",
    "Clarksville Hope Riders Biker Church",
    "Cleveland First",
    "Columbia First",
    "Columbia Grace",
    "Columbus First",
    "Community Fellowship",
    "Concord Community",
    "Covington",
    "Crossroads",
    "Crossroads Community",
    "Crystal Springs",
    "DaySpring Community",
    "Dickson Lighthouse",
    "Donelson",
    "Dover",
    "Elmore",
    "Erin",
    "Fellowship",
    "Fly",
    "Foundry",
    "Franklin Community Faith",
    "Fulton First",
    "Gallatin First",
    "Gateway Community",
    "Glencliff Hispanic",
    "Gloster",
    "Goodlettsville",
    "Gray's Chapel",
    "Grenada First",
    "Griffin's Chapel",
    "Gulf Coast Family",
    "Harmony",
    "Hattiesburg First",
    "Hazkehurst",
    "Hendersonville",
    "Hermitage",
    "Highland",
    "Hillside Community",
    "Igelsia Del Nazareno Eben-Ezer",
    "Igelsia Hispana Betel de Gallatin",
    "Jason Chapel",
    "Joelton",
    "Jones Chapel",
    "Kingston Springs",
    "Laurel Bresee",
    "Laurel First",
    "Lewisburg First",
    "Libertad en Cristo",
    "Lifepointe Community",
    "Long Creek",
    "Madison",
    "Madison Hispanic",
    "Magnolia",
    "Memphis Calvary",
    "Memphis Eastside",
    "Memphis Emmanuel",
    "Memphis Friendship",
    "Memphis Grace",
    "Memphis Holiness Tabernacle",
    "Memphis Hope Tabernacle",
    "Memphis Refreshing Springs",
    "Meridian Fitkins Memorial",
    "Meridian Northside",
    "Millington",
    "Mount Wesley-Akin Chapel",
    "Nashville Blakemore",
    "Nashville Christ",
    "Nashville First (Main Campus)",
    "Nashville First (Bethel Campus)",
    "Nashville First (Calvary) Korean",
    "Nashville Grace",
    "Nashville Inglewood",
    "Nashville Kaleo",
    "Nashville Trinity",
    "New Albany First",
    "New Vision",
    "Old Hickory",
    "Oxford",
    "Palabra de Vida",
    "Paris",
    "Park Lane",
    "Pearson Chapel",
    "Pine Hill",
    "Pleasant Point",
    "Portland",
    "Prentiss",
    "Quitman",
    "Ripley Hispanic",
    "Rosebloom",
    "Savannah",
    "Sentobia N.O. Nabors Memorial",
    "Spring Hill Restoration Community",
    "Springfield First",
    "Starkville Wesley Community",
    "Trevecca Community",
    "Union",
    "Vicksburg First/Vicksburg Hispanic",
    "Victory Praise and Worship Center",
    "Waverly",
    "Way of the Cross",
    "Waynesboro Trinity",
    'White House',
    "Westwind Community of Faith",
    "Woodlawn Community",
    "Individual - No church affiliation",
    "Other"
]

In [3]:

files = os.listdir(file_root)
files = [f for f in files if os.path.isfile(f'{file_root}/{f}') and f != '.DS_Store'] 
files

['Camp_Registration_-_20242024-06-01_10_25_17.csv']

In [4]:
def create_directory(process):
    outdir = f'{root_dir}/{process}/{date.today()}/'
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [5]:
def write_csv(df, df_name, process, filename):
    create_directory(process)
    df.to_csv(f'{date.today()}_{filename}.csv')

In [6]:
def write_excel(df, df_name, process, filename, indexed=False):
    create_directory(process)
    df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [7]:
def move_to_processed(filename):
    src_file = f'{file_root}/{filename}'
    dst_file = f'{root_dir}/processed/{filename}'
    
    shutil.move(src_file, dst_file)

In [8]:
def strip_filename_from_download_date(filename):
    filename_parts = filename.split('_-_')
    return filename_parts[0]

strip_filename_from_download_date(files[0])

'Camp_Registration'

In [9]:
def get_price(row):
    if row['Are you a student or a chaperone'] == 'Chaperone':
        return pricing_breakdown['chaperone'][row['At which camp will you be a chaperone?']]
    
    submission_date_string = row['Date']
    price_dates = pricing_breakdown['student'].keys()
    max_date = datetime.strptime(list(price_dates)[-1], '%m-%d-%y')
    
    for price_date_string in price_dates:
        submission_date = datetime.strptime(submission_date_string, '%b %d, %Y')
        price_date = datetime.strptime(price_date_string, '%m-%d-%y')
        if submission_date <= price_date:
            return pricing_breakdown['student'][price_date_string]
        
        elif submission_date > max_date:
            return pricing_breakdown['student'][list(price_dates)[-1]] + late_fee

In [10]:
# def camp
def create_church_info_sheets(file, filename):
    df_original = pd.read_csv(f'{file_root}/{file}')

    df = df_original.copy(deep=True)
    # df['Camp'] = df.apply(lambda x: x['At which camp will you be a chaperone?'] if x['At which camp will you be a chaperone?'] else x['Which Camp?'])
    df['Camp'] = df['At which camp will you be a chaperone?'].combine_first(df['Which Camp?'])

    cols = df_original.columns
    cols = [*cols, 'Camp']
    df = df.loc[:, cols].fillna('')
    df = df.groupby(['What church are you a part of?', 'Last Name', 'First Name'], group_keys=False).apply(lambda x: x)
    
    df.reset_index(inplace=True)
    
    # df['What church are you a part of?'] = df['What church are you a part of?'].apply(lambda x: x.strip())
    for church in church_list:
        df_church = df.loc[df['What church are you a part of?'].str.strip() == church]
        
        if not df_church.empty:
            df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
            df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
            df_church['Total Due'] = df_church.apply(lambda x: x['Price'] - x['Paid'], axis=1)
            df_church.rename(columns = {"First Name.1": "Guardian First Name", "Last Name.1": "Guardian Last Name","First Name.2": "Youth Leader First Name", "Last Name.2": "Youth Leader Last Name"}, inplace = True)
          
            df_church = df_church.groupby(['Camp'], group_keys=False).apply(lambda x: x)
            df_church.sort_values(['Submission Date', 'Last Name', 'First Name'], inplace=True)
            df_church.reset_index(inplace=True)

            df_church = df_church.drop(columns=['level_0', 'index'])

            df_church.loc['Total',:] = df_church.sum(axis=0, numeric_only=True)

            
            export_path = f"{church_export_root}/{date.today()}/{date.today()}_{church.replace(' ', '_').lower()}"
            write_excel(df_church, church, 'church', export_path)


# create_church_info_sheets(files[0], strip_filename_from_download_date(files[0]))

In [11]:
def build_combined_spreadsheet():
    #TODO: Create New File

    columns_student_worksheet = [
        'Last Name',
        'First Name', #combine to name
        'Church',
        'Shirt Size',
        'Teen/Male',
        'Teen/Female',
        'Adult/Male',
        'Adult/Female',
        'Early Bird',
        'Standard',
        'Late Fee',
        'Adult Fee',
        'Form of Payment/Check #',
        'Amount Owed',
        'Amount Paid',
        'Balance',
    ]

    columns_church_worksheet = [
        'Church Name',
        'Amount Paid',
        'Teen/Male',
        'Teen/Female',
        'Adult/Male',
        'Adult/Female',
        'Amount Balance',
        'Amount Paid',
        'Totals',
        'Small',
        'Medium',
        'Large',
        'XL',
        '2X',
        '3X',
        '4X',
        '5X',
    ]

    create_directory(process)
    df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [12]:
for file in files:
    filename = strip_filename_from_download_date(file)

    create_church_info_sheets(file, filename)
    # build_combined_spreadsheet()
    move_to_processed(file)

/var/folders/zk/c73ns6dn17b9_9v4rlbyrwxc0000gn/T/ipykernel_13402/1301334253.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['What church are you a part of?', 'Last Name', 'First Name'], group_keys=False).apply(lambda x: x)
/var/folders/zk/c73ns6dn17b9_9v4rlbyrwxc0000gn/T/ipykernel_13402/1301334253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
/var/folders/zk/c7